# Using onset/duration format

Many datasets store interval annotations as onset + duration rather than explicit interval boundaries. This notebook shows how to use `DimensionInterval` with onset/duration coordinates directly, without needing to construct `pd.IntervalIndex` objects.

## Why onset/duration?

Common data formats like TextGrid, Praat, and many annotation tools export intervals as:
- `onset`: when the interval starts
- `duration`: how long the interval lasts

With `DimensionInterval`, you can use these coordinates directly.

In [1]:
import xarray as xr
from linked_indices import DimensionInterval, example_data

## Loading annotation data

Annotation data typically comes as a pandas DataFrame with onset, duration, and label columns. Let's load some example speech annotation data:

In [2]:
# Load example speech annotations (simulating data from Praat, TextGrid, etc.)
annotations = example_data.speech_annotations()
annotations

,onset,duration,word
0,0.5,1.2,hello
1,2.1,1.8,world
2,4.5,2.0,how
3,7.0,2.5,are


Notice that the annotations have **gaps** between them - this is common in real speech data where there are pauses between words. For example, "hello" ends at 1.7s but "world" doesn't start until 2.1s.

In [3]:
# Generate a simulated audio signal
times, audio_signal = example_data.generate_audio_signal(duration=10.0)
print(f"Audio: {len(times)} samples over {times[-1]:.1f} seconds")

Audio: 1000 samples over 10.0 seconds


## Converting DataFrame to xarray coordinates

To use the annotations with `DimensionInterval`, we need to convert the DataFrame columns to xarray coordinates. Each column becomes a coordinate on the annotation dimension:

In [4]:
# Create xarray Dataset with audio data and annotation coordinates
ds = xr.Dataset(
    {"audio": (("time",), audio_signal)},
    coords={
        # Continuous time dimension
        "time": times,
        # Annotation coordinates from the DataFrame
        "word_onset": ("word", annotations["onset"].values),
        "word_duration": ("word", annotations["duration"].values),
        "word": ("word", annotations["word"].values),
    },
)
ds

<xarray.Dataset> Size: 16kB
Dimensions:        (time: 1000, word: 4)
Coordinates:
  * time           (time) float64 8kB 0.0 0.01001 0.02002 ... 9.98 9.99 10.0
  * word           (word) object 32B 'hello' 'world' 'how' 'are'
    word_onset     (word) float64 32B 0.5 2.1 4.5 7.0
    word_duration  (word) float64 32B 1.2 1.8 2.0 2.5
Data variables:
    audio          (time) float64 8kB 0.09934 0.3325 0.7231 ... -0.4744 0.1145

## Applying the DimensionInterval index

To use onset/duration format, pass the `onset_duration_coords` option mapping dimension names to `(onset_coord, duration_coord)` tuples:

In [5]:
ds = ds.drop_indexes(["time", "word"]).set_xindex(
    ["time", "word_onset", "word_duration", "word"],
    DimensionInterval,
    onset_duration_coords={"word": ("word_onset", "word_duration")},
)
ds

<xarray.Dataset> Size: 16kB
Dimensions:        (time: 1000, word: 4)
Coordinates:
  * time           (time) float64 8kB 0.0 0.01001 0.02002 ... 9.98 9.99 10.0
  * word           (word) object 32B 'hello' 'world' 'how' 'are'
  * word_onset     (word) float64 32B 0.5 2.1 4.5 7.0
  * word_duration  (word) float64 32B 1.2 1.8 2.0 2.5
Data variables:
    audio          (time) float64 8kB 0.09934 0.3325 0.7231 ... -0.4744 0.1145
Indexes:
  ┌ time           DimensionInterval
  │ word_onset
  │ word_duration
  └ word

Notice that:
- The `word_onset` and `word_duration` coordinates are still visible
- No synthetic `word_intervals` coordinate was created
- All coordinates are linked under a single `DimensionInterval` index

## Selecting data

Selection works exactly the same as with the IntervalIndex format. When you select on any dimension, all other dimensions are automatically constrained.

In [6]:
# Select by word label - time is automatically constrained
hello_data = ds.sel(word="hello")
print(
    f"Word 'hello' spans time {hello_data.time.values.min():.2f} to {hello_data.time.values.max():.2f}"
)
hello_data

Word 'hello' spans time 0.50 to 1.69


<xarray.Dataset> Size: 2kB
Dimensions:        (time: 120, word: 1)
Coordinates:
  * time           (time) float64 960B 0.5005 0.5105 0.5205 ... 1.682 1.692
  * word           (word) object 8B 'hello'
  * word_onset     (word) float64 8B 0.5
  * word_duration  (word) float64 8B 1.2
Data variables:
    audio          (time) float64 960B 0.06827 -0.0255 -0.1359 ... 0.4946 0.2634
Indexes:
  ┌ time           DimensionInterval
  │ word_onset
  │ word_duration
  └ word

In [7]:
# Select by time range - words are automatically constrained
time_slice = ds.sel(time=slice(2, 5))
print(f"Words in time range [2, 5]: {list(time_slice.word.values)}")
time_slice

Words in time range [2, 5]: ['world', 'how']


<xarray.Dataset> Size: 5kB
Dimensions:        (time: 300, word: 2)
Coordinates:
  * time           (time) float64 2kB 2.002 2.012 2.022 ... 4.975 4.985 4.995
  * word           (word) object 16B 'world' 'how'
  * word_onset     (word) float64 16B 2.1 4.5
  * word_duration  (word) float64 16B 1.8 2.0
Data variables:
    audio          (time) float64 2kB 0.1481 0.5324 0.8344 ... -0.6739 -0.4654
Indexes:
  ┌ time           DimensionInterval
  │ word_onset
  │ word_duration
  └ word

In [8]:
# Select by onset value
ds.sel(word_onset=4.5)

<xarray.Dataset> Size: 3kB
Dimensions:        (time: 200, word: 1)
Coordinates:
  * time           (time) float64 2kB 4.505 4.515 4.525 ... 6.476 6.486 6.496
  * word           (word) object 8B 'how'
  * word_onset     (word) float64 8B 4.5
  * word_duration  (word) float64 8B 2.0
Data variables:
    audio          (time) float64 2kB 0.01689 0.2374 -0.2569 ... -0.0957 0.1158
Indexes:
  ┌ time           DimensionInterval
  │ word_onset
  │ word_duration
  └ word

## Handling gaps

Our word annotations have gaps between them (silence between words). Let's see what happens when we select time in a gap:

In [9]:
# Time 1.8 to 2.0 is in the gap between "hello" (ends at 1.7) and "world" (starts at 2.1)
gap_selection = ds.sel(time=slice(1.75, 2.0))
print(f"Time points selected: {len(gap_selection.time)}")
print(f"Words selected: {list(gap_selection.word.values)}")

Time points selected: 25
Words selected: ['hello']


## Multiple onset/duration dimensions

You can have multiple interval dimensions, each with their own onset/duration coordinates. This is common for hierarchical annotations like words and phonemes:

In [10]:
# Load multi-level annotations (words and phonemes)
word_annotations, phoneme_annotations = example_data.multi_level_annotations()

print("Word annotations:")
display(word_annotations)

print("\nPhoneme annotations:")
display(phoneme_annotations)

Word annotations:


,onset,duration,word,part_of_speech
0,0.0,2.5,hello,interjection
1,3.0,2.5,world,noun
2,6.0,3.5,test,noun



Phoneme annotations:


,onset,duration,phoneme
0,0.0,0.8,hh
1,0.8,0.9,eh
2,1.7,0.8,l
3,3.0,0.9,w
4,3.9,0.8,er
5,4.7,0.8,ld
6,6.0,1.2,t
7,7.2,2.3,st


In [11]:
# Generate audio signal
times, audio = example_data.generate_audio_signal(duration=10.0)

# Create dataset with both word and phoneme annotations from DataFrames
ds_multi = xr.Dataset(
    {"audio": (("time",), audio)},
    coords={
        "time": times,
        # Word annotations
        "word_onset": ("word", word_annotations["onset"].values),
        "word_duration": ("word", word_annotations["duration"].values),
        "word": ("word", word_annotations["word"].values),
        "part_of_speech": ("word", word_annotations["part_of_speech"].values),
        # Phoneme annotations
        "phoneme_onset": ("phoneme", phoneme_annotations["onset"].values),
        "phoneme_duration": ("phoneme", phoneme_annotations["duration"].values),
        "phoneme": ("phoneme", phoneme_annotations["phoneme"].values),
    },
)

# Apply index with both onset/duration mappings
ds_multi = ds_multi.drop_indexes(["time", "word", "phoneme"]).set_xindex(
    [
        "time",
        "word_onset",
        "word_duration",
        "word",
        "part_of_speech",
        "phoneme_onset",
        "phoneme_duration",
        "phoneme",
    ],
    DimensionInterval,
    onset_duration_coords={
        "word": ("word_onset", "word_duration"),
        "phoneme": ("phoneme_onset", "phoneme_duration"),
    },
)
ds_multi

<xarray.Dataset> Size: 16kB
Dimensions:           (time: 1000, word: 3, phoneme: 8)
Coordinates:
  * time              (time) float64 8kB 0.0 0.01001 0.02002 ... 9.98 9.99 10.0
  * word              (word) object 24B 'hello' 'world' 'test'
  * phoneme           (phoneme) object 64B 'hh' 'eh' 'l' 'w' 'er' 'ld' 't' 'st'
  * word_onset        (word) float64 24B 0.0 3.0 6.0
  * word_duration     (word) float64 24B 2.5 2.5 3.5
  * part_of_speech    (word) object 24B 'interjection' 'noun' 'noun'
  * phoneme_onset     (phoneme) float64 64B 0.0 0.8 1.7 3.0 3.9 4.7 6.0 7.2
  * phoneme_duration  (phoneme) float64 64B 0.8 0.9 0.8 0.9 0.8 0.8 1.2 2.3
Data variables:
    audio             (time) float64 8kB 0.09934 0.3325 ... -0.4744 0.1145
Indexes:
  ┌ time              DimensionInterval
  │ word_onset
  │ word_duration
  │ word
  │ part_of_speech
  │ phoneme_onset
  │ phoneme_duration
  └ phoneme

In [12]:
# Select word "hello" - both time AND phonemes are constrained
hello = ds_multi.sel(word="hello")
print("Word: hello")
print(f"Time range: {hello.time.values.min():.2f} - {hello.time.values.max():.2f}")
print(f"Phonemes: {list(hello.phoneme.values)}")

Word: hello
Time range: 0.00 - 2.49
Phonemes: ['hh', 'eh', 'l']


In [13]:
# Select by part of speech - finds all nouns
nouns = ds_multi.sel(part_of_speech="noun")
print(f"Nouns: {list(nouns.word.values)}")

Nouns: ['world', 'test']


## Controlling interval closedness

By default, intervals are left-closed `[onset, onset+duration)`. You can change this with the `interval_closed` option:

In [14]:
# Reload data for fresh example
annotations = example_data.speech_annotations()
times, audio = example_data.generate_audio_signal()

# Create with right-closed intervals (onset, onset+duration]
ds_right = xr.Dataset(
    {"audio": (("time",), audio)},
    coords={
        "time": times,
        "word_onset": ("word", annotations["onset"].values),
        "word_duration": ("word", annotations["duration"].values),
        "word": ("word", annotations["word"].values),
    },
)

ds_right = ds_right.drop_indexes(["time", "word"]).set_xindex(
    ["time", "word_onset", "word_duration", "word"],
    DimensionInterval,
    onset_duration_coords={"word": ("word_onset", "word_duration")},
    interval_closed="right",  # Options: "left", "right", "both", "neither"
)
print("Created dataset with right-closed intervals (onset, onset+duration]")

Created dataset with right-closed intervals (onset, onset+duration]


## Using the library helper

The `intervals_from_dataframe` function in `example_data` provides a convenient way to convert annotation DataFrames to xarray Datasets with properly named coordinates:

In [15]:
from linked_indices.example_data import intervals_from_dataframe

# Convert word annotations DataFrame to xarray Dataset
word_ds = intervals_from_dataframe(word_annotations, dim_name="word", label_col="word")
print("Word dataset:")
print(word_ds)

# Convert phoneme annotations DataFrame to xarray Dataset
phoneme_ds = intervals_from_dataframe(
    phoneme_annotations, dim_name="phoneme", label_col="phoneme"
)
print("\nPhoneme dataset:")
print(phoneme_ds)

Word dataset:
<xarray.Dataset> Size: 96B
Dimensions:         (word: 3)
Coordinates:
  * word            (word) object 24B 'hello' 'world' 'test'
    word_onset      (word) float64 24B 0.0 3.0 6.0
    word_duration   (word) float64 24B 2.5 2.5 3.5
    part_of_speech  (word) object 24B 'interjection' 'noun' 'noun'
Data variables:
    *empty*

Phoneme dataset:
<xarray.Dataset> Size: 192B
Dimensions:           (phoneme: 8)
Coordinates:
  * phoneme           (phoneme) object 64B 'hh' 'eh' 'l' 'w' 'er' 'ld' 't' 'st'
    phoneme_onset     (phoneme) float64 64B 0.0 0.8 1.7 3.0 3.9 4.7 6.0 7.2
    phoneme_duration  (phoneme) float64 64B 0.8 0.9 0.8 0.9 0.8 0.8 1.2 2.3
Data variables:
    *empty*


In [16]:
# Merge annotation datasets and add time/audio data
times, audio = example_data.generate_audio_signal(duration=10.0)

ds_from_helper = xr.merge([word_ds, phoneme_ds])
ds_from_helper["audio"] = (("time",), audio)
ds_from_helper = ds_from_helper.assign_coords(time=times)

# Apply the DimensionInterval index
ds_from_helper = ds_from_helper.drop_indexes(["time", "word", "phoneme"]).set_xindex(
    [
        "time",
        "word_onset",
        "word_duration",
        "word",
        "part_of_speech",
        "phoneme_onset",
        "phoneme_duration",
        "phoneme",
    ],
    DimensionInterval,
    onset_duration_coords={
        "word": ("word_onset", "word_duration"),
        "phoneme": ("phoneme_onset", "phoneme_duration"),
    },
)
ds_from_helper

<xarray.Dataset> Size: 16kB
Dimensions:           (word: 3, phoneme: 8, time: 1000)
Coordinates:
  * word              (word) object 24B 'hello' 'world' 'test'
  * phoneme           (phoneme) object 64B 'hh' 'eh' 'l' 'w' 'er' 'ld' 't' 'st'
  * time              (time) float64 8kB 0.0 0.01001 0.02002 ... 9.98 9.99 10.0
  * word_onset        (word) float64 24B 0.0 3.0 6.0
  * word_duration     (word) float64 24B 2.5 2.5 3.5
  * part_of_speech    (word) object 24B 'interjection' 'noun' 'noun'
  * phoneme_onset     (phoneme) float64 64B 0.0 0.8 1.7 3.0 3.9 4.7 6.0 7.2
  * phoneme_duration  (phoneme) float64 64B 0.8 0.9 0.8 0.9 0.8 0.8 1.2 2.3
Data variables:
    audio             (time) float64 8kB 0.09934 0.3325 ... -0.4744 0.1145
Indexes:
  ┌ time              DimensionInterval
  │ word_onset
  │ word_duration
  │ word
  │ part_of_speech
  │ phoneme_onset
  │ phoneme_duration
  └ phoneme

## Summary

The onset/duration format provides a convenient way to work with interval data without manually constructing `pd.IntervalIndex` objects:

1. **Load annotations** as a pandas DataFrame (from TextGrid, Praat, CSV, etc.)
2. **Convert to coordinates** using helper functions or manual assignment
3. **Apply the index** with `onset_duration_coords` mapping
4. **Select data** - all selection operations work identically to IntervalIndex format

### Helper functions

| Function | Use case |
|----------|----------|
| `intervals_from_dataframe()` | Convert a single-event-type DataFrame |
| `intervals_from_long_dataframe()` | Convert a multi-event-type DataFrame with category column |

### Key features

- **Natural representation**: Use onset + duration directly from your data files
- **Library helpers**: Handle coordinate naming conventions automatically
- **Visible coordinates**: onset and duration remain as regular coordinates  
- **Full functionality**: All selection operations work identically
- **Multiple dimensions**: Support for multiple onset/duration pairs
- **Gap support**: Non-contiguous intervals work correctly
- **Mixed events**: Handle DataFrames with multiple event types

## Handling multiple event types in one DataFrame

Sometimes annotation data comes as a single "long format" DataFrame with multiple event types (words, phonemes, stimuli, etc.) distinguished by a category column. The `intervals_from_long_dataframe` function handles this case:

In [17]:
# Load example mixed-event annotations
mixed_df = example_data.mixed_event_annotations()
mixed_df

,onset,duration,label,event_type
0,0.0,2.5,hello,word
1,3.0,2.5,world,word
2,6.0,3.5,test,word
3,0.0,0.8,hh,phoneme
4,0.8,0.9,eh,phoneme
5,1.7,0.8,l,phoneme
6,3.0,0.9,w,phoneme
7,3.9,1.6,orld,phoneme
8,0.0,5.0,image_A,stimulus
9,5.0,5.0,image_B,stimulus


In [18]:
from linked_indices.example_data import intervals_from_long_dataframe

# Convert all event types at once
interval_ds = intervals_from_long_dataframe(mixed_df)
print("Dimensions:", list(interval_ds.dims))
print("Coordinates:", list(interval_ds.coords))
interval_ds

Dimensions: ['word', 'phoneme', 'stimulus']
Coordinates: ['word_onset', 'word_duration', 'word', 'phoneme_onset', 'phoneme_duration', 'phoneme', 'stimulus_onset', 'stimulus_duration', 'stimulus']


<xarray.Dataset> Size: 240B
Dimensions:            (word: 3, phoneme: 5, stimulus: 2)
Coordinates:
  * word               (word) object 24B 'hello' 'world' 'test'
  * phoneme            (phoneme) object 40B 'hh' 'eh' 'l' 'w' 'orld'
  * stimulus           (stimulus) object 16B 'image_A' 'image_B'
    word_onset         (word) float64 24B 0.0 3.0 6.0
    word_duration      (word) float64 24B 2.5 2.5 3.5
    phoneme_onset      (phoneme) float64 40B 0.0 0.8 1.7 3.0 3.9
    phoneme_duration   (phoneme) float64 40B 0.8 0.9 0.8 0.9 1.6
    stimulus_onset     (stimulus) float64 16B 0.0 5.0
    stimulus_duration  (stimulus) float64 16B 5.0 5.0
Data variables:
    *empty*

In [19]:
# Add time/audio and apply DimensionInterval
times, audio = example_data.generate_audio_signal(duration=10.0)

ds_mixed = interval_ds.copy()
ds_mixed["audio"] = (("time",), audio)
ds_mixed = ds_mixed.assign_coords(time=times)

# Apply the index with all three event types
ds_mixed = ds_mixed.drop_indexes(["time", "word", "phoneme", "stimulus"]).set_xindex(
    [
        "time",
        "word_onset",
        "word_duration",
        "word",
        "phoneme_onset",
        "phoneme_duration",
        "phoneme",
        "stimulus_onset",
        "stimulus_duration",
        "stimulus",
    ],
    DimensionInterval,
    onset_duration_coords={
        "word": ("word_onset", "word_duration"),
        "phoneme": ("phoneme_onset", "phoneme_duration"),
        "stimulus": ("stimulus_onset", "stimulus_duration"),
    },
)
ds_mixed

<xarray.Dataset> Size: 16kB
Dimensions:            (word: 3, phoneme: 5, stimulus: 2, time: 1000)
Coordinates:
  * word               (word) object 24B 'hello' 'world' 'test'
  * phoneme            (phoneme) object 40B 'hh' 'eh' 'l' 'w' 'orld'
  * stimulus           (stimulus) object 16B 'image_A' 'image_B'
  * time               (time) float64 8kB 0.0 0.01001 0.02002 ... 9.98 9.99 10.0
  * word_onset         (word) float64 24B 0.0 3.0 6.0
  * word_duration      (word) float64 24B 2.5 2.5 3.5
  * phoneme_onset      (phoneme) float64 40B 0.0 0.8 1.7 3.0 3.9
  * phoneme_duration   (phoneme) float64 40B 0.8 0.9 0.8 0.9 1.6
  * stimulus_onset     (stimulus) float64 16B 0.0 5.0
  * stimulus_duration  (stimulus) float64 16B 5.0 5.0
Data variables:
    audio              (time) float64 8kB 0.09934 0.3325 ... -0.4744 0.1145
Indexes:
  ┌ time               DimensionInterval
  │ word_onset
  │ word_duration
  │ word
  │ phoneme_onset
  │ phoneme_duration
  │ phoneme
  │ stimulus_onset
  │ stimulus_duration
  └ stimulus

In [20]:
# Selecting a stimulus constrains words and phonemes too
result = ds_mixed.sel(stimulus="image_A")
print("Stimulus: image_A [0.0, 5.0)")
print(f"Words in range: {list(result.word.values)}")
print(f"Phonemes in range: {list(result.phoneme.values)}")

Stimulus: image_A [0.0, 5.0)
Words in range: ['hello', 'world']
Phonemes in range: ['hh', 'eh', 'l', 'w', 'orld']


### Manual iteration for selective event types

If you only want some event types, you can filter and apply `intervals_from_dataframe` iteratively:

In [21]:
# Only include words and phonemes (exclude stimuli)
datasets = []
for event_type in ["word", "phoneme"]:  # selective event types
    subset = mixed_df[mixed_df["event_type"] == event_type].drop(columns=["event_type"])
    ds_subset = intervals_from_dataframe(subset, dim_name=event_type, label_col="label")
    datasets.append(ds_subset)

ds_selective = xr.merge(datasets)
print("Dimensions:", list(ds_selective.dims))  # No 'stimulus' dimension

Dimensions: ['word', 'phoneme']
